# TescanDataReader Demo

Note: this demo is a work in progress and was created for the purpose of demonstrating a contibution for the reader hackathon!

The TescanDataReader in its current state allows:

- Reading of geometry
- Reading of dark fields
- Reading of flat fields
- Reading of projections 
    - Reading unnormalised data
    - Reading Flipped Data
    - Cropping Data




## CIL Version

This notebook was developed using CIL v25.0.0

Data Comes as Tiff files. Data acquisition settings is stored in the Acquisiton Settings XRE.txt file

## Dataset
place dataset in here

Update this filepath to where you have saved the dataset:

In [ ]:
file_path = r'Acquisition settings XRE.txt'

## Loading Geometry

In [ ]:
from readers.TescanDataReader import TescanDataReader

from cil.utilities.display import show2D, show_geometry

In [ ]:
reader = TescanDataReader(file_path)

In [ ]:
acq_geom = reader.get_geometry()

show_geometry(acq_geom)

In [ ]:
print(acq_geom)

## Loading Projections, Flat and Dark Fields

In [ ]:
acq_data = reader.read()

show2D(acq_data, origin='upper-left')

In [ ]:
dark_field = reader.load_darkfield()

show2D(dark_field, origin='upper-left')

In [ ]:
flat_field = reader.load_flatfield()
show2D(flat_field, origin='upper-left')

# Pre-processing and Reconstruction

For details of the pre-processing steps performed, see our [CIL-Demos example which uses the same dataset](https://github.com/TomographicImaging/CIL-Demos/blob/main/demos/1_Introduction/03_preprocessing.ipynb)! However, in the CIL-demo the data had been converted into the CIL Nexus format first.

In [ ]:
from cil.processors import TransmissionAbsorptionConverter, Slicer, CentreOfRotationCorrector
from cil.recon import FDK
import numpy as np

data_norm = reader.read()

data_exp = TransmissionAbsorptionConverter()(data_norm)

data_crop = Slicer(roi={'vertical': (10, np.shape(data_exp)[0]-10)})(data_exp)

data_cor = CentreOfRotationCorrector.image_sharpness(backend='tigre', search_range=100, tolerance=0.1)(data_crop)

recon = FDK(data_cor).run()

In [ ]:
show2D(recon,slice_list=('horizontal_y', 70), title='Reconstruction FDK',  origin='upper-left')